# Centralised Logistic Regression Classifier
Author: **Giorgio Polla**

In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [2]:
def read_file(filename):
    read_data = pd.read_csv(filename, header=None)
    return read_data

In [3]:
df = read_file("../data/spam.csv")
df.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [4]:
def get_x(df):
    return df.iloc[:, 0:-1].values

def get_y(df):
    return df.iloc[:, -1].values

def standardize(df):
    x = get_x(df)
    df_y = df.iloc[:, 57]

    standard_scaler = preprocessing.StandardScaler()
    x_scaled = standard_scaler.fit_transform(x)

    df_x = pd.DataFrame(x_scaled)
    df_scaled = df_x.join(df_y)

    return df_scaled

In [5]:
df = standardize(df)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,-0.342434,0.330885,0.712859,-0.0469,0.011565,-0.350266,-0.291794,-0.262562,-0.323302,-0.371364,...,-0.158453,-0.514307,-0.155198,0.624007,-0.308355,-0.103048,-0.045247,0.045298,-0.008724,1
1,0.345359,0.051909,0.435130,-0.0469,-0.256117,0.672399,0.244743,-0.088010,-0.323302,1.086711,...,-0.158453,-0.026007,-0.155198,0.126203,0.423783,0.008763,-0.002443,0.250563,1.228324,1
2,-0.145921,-0.165072,0.851723,-0.0469,1.364846,0.343685,0.193644,0.036670,1.974017,0.016422,...,-0.117376,0.014684,-0.155198,0.008496,0.440053,-0.079754,0.145921,2.221106,3.258733,1
3,-0.342434,-0.165072,-0.556761,-0.0469,0.472573,-0.350266,0.500237,1.308402,0.789462,0.605857,...,-0.158453,-0.007511,-0.155198,-0.161934,-0.308355,-0.103048,-0.052150,-0.062466,-0.152222,1
4,-0.342434,-0.165072,-0.556761,-0.0469,0.472573,-0.350266,0.500237,1.308402,0.789462,0.605857,...,-0.158453,-0.014910,-0.155198,-0.164387,-0.308355,-0.103048,-0.052150,-0.062466,-0.152222,1


In [6]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def loss(p, y, w, lambda_reg):
    epsilon = 1e-10
    
    error_loss = -np.average(
        y * np.log(p + epsilon) + (1 - y) * np.log(1 - p + epsilon)
    )
    reg_loss = lambda_reg * np.sum(np.square(w)) / (2 * y.size)
    
    return error_loss + reg_loss

def predict_probability(x, b, w):
    z = np.dot(x, w) + b
    return sigmoid(z)

In [7]:
def train(df, iterations=1000, learning_rate=10, lambda_reg=0.01, verbose=True):
    x = get_x(df)
    y = get_y(df)

    w_train = np.zeros(x.shape[1])
    b = 0
    
    loss_array = []

    for it in range(iterations):
        predictions = predict_probability(x, b, w_train)
        
        loss_a = loss(predictions, y, w_train, lambda_reg)
        
        gradient_w = np.dot(x.T, (predictions - y))
        gradient_b = np.average(predictions - y)
        regularization = lambda_reg * w_train

        w_train -= learning_rate * (gradient_w + regularization) / y.size
        b -= learning_rate * gradient_b

        if verbose and (it % (iterations/5) == 0 or it+1 == iterations):
            print("It. %4d\t|\tLoss: %0.4f" % 
                  (it, loss_a)
                 )
    
        if it % 10 == 0:
            loss_array.append(
                loss_a
            )
            
    return b, w_train

In [8]:
def predict(x, b, w, threshold):
    prob = predict_probability(x, b, w)
    return prob >= threshold

def accuracy(df, b, w, threshold=0.5):
    x = get_x(df)
    y = get_y(df)
    predictions = predict(x, b, w, threshold=threshold)

    acc = np.average(predictions == y)
    
    return acc

def get_block_data(df, fold, tot_folds):
    fold_size = math.floor(df.shape[0] / tot_folds)

    start_index = fold_size * fold
    end_index = start_index + fold_size

    df_test = df.loc[start_index:end_index]
    df.drop(df.loc[start_index:end_index].index, inplace=True)

    return df, df_test

def shuffle(df):
    return df.sample(frac=1).reset_index(drop=True)

In [9]:
def cross_validation(df, iterations=1000, learning_rate=10, threshold=0.5,
                     lambda_reg=0.1, folds=10, verbose_train=False):
    avg_acc = 0
    df = shuffle(df)

    for i in range(folds):
        print("Fold number " + str(i+1))
        tr_data, test_data = get_block_data(df.copy(), i, folds)
        b, w = train(
            tr_data,
            learning_rate=learning_rate,
            lambda_reg=lambda_reg,
            verbose=verbose_train
        )
        avg_acc += accuracy(test_data, b, w, threshold=threshold)

    avg_acc /= folds

    print("\nAVG acc: %0.4f" % avg_acc)
    return avg_acc

In [10]:
cross_validation(df,
                 folds=10,
                 verbose_train=False,
                 iterations=1000,
                 learning_rate=10,
                 lambda_reg=0.1,
                 threshold=0.5
                )

Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9254


0.9253796095444684

In [13]:
b, w = train(df,
             iterations=1000, 
             learning_rate=10, 
             lambda_reg=0.1
            )
print(b)
accuracy(df, b, w)

It.    0	|	Loss: 0.6931
It.   20	|	Loss: 0.2189
It.   40	|	Loss: 0.2148
It.   60	|	Loss: 0.2132
It.   80	|	Loss: 0.2124
It.   99	|	Loss: 0.2119
-2.3069801718858294


0.9271897413605737

### Skopt analysis
Not required.

In [41]:
import skopt
SPACE = [skopt.space.Real(
        0.001, 
        10, 
        name = 'lambda_reg', 
        prior='log_uniform'
    ),
    skopt.space.Real(
        0.01, 
        10, 
        name='learning_rate', 
        prior='log_uniform'
    )   
]

@skopt.utils.use_named_args(SPACE)
def objective(**params):
    all_params = {**params}
    return -1.0 * cross_validation(df, all_params)

random_results = skopt.dummy_minimize(objective, SPACE)
forest_results = skopt.forest_minimize(objective, SPACE)
gbrt_results = skopt.gbrt_minimize(objective, SPACE)
gp_results = skopt.gp_minimize(objective, SPACE)

results = [('random_results', random_results),
           ('forest_results', forest_results),
           ('gbrt_results', gbrt_results),
           ('gp_results', gp_results)]

Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9254
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9271
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9265
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9256
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9252
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9258
Fold number 1
Fold number 2
Fold number 3
Fold numbe


AVG acc: 0.9254
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9245
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9252
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9252
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9252
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9249
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9254
Fold number 1
Fold number 2
Fold nu

Fold number 10

AVG acc: 0.9241
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9247
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9252
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9234
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9249
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9256
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9256
Fold number 1
Fold n

Fold number 9
Fold number 10

AVG acc: 0.9254
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9252
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9256
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9254
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9256
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9265
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9254
Fold n

Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9258
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9262
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9243
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9256
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9223
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9254
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc:

...

Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9241


/home/giorgio/anaconda3/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9245
Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9236


/home/giorgio/anaconda3/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fold number 1
Fold number 2
Fold number 3
Fold number 4
Fold number 5
Fold number 6
Fold number 7
Fold number 8
Fold number 9
Fold number 10

AVG acc: 0.9278


End.